In [38]:
import openai
from tqdm.auto import tqdm
import os
import json
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [36]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cpu


In [11]:
openai_api_key = os.environ.get("OPENAI_API_KEY")
openai_api_base = os.environ.get("OPENAI_API_BASE")

In [31]:
def get_single_answer_gpt(model_id, prompt):
    for i in range(10):
        try:
            completion = openai.ChatCompletion.create(
                model = model_id,
                messages = [{"role": "user", "content": prompt}],
                stream = False,
            ).choices[0].message.content
            break
        except Exception as e:
            completion = f"AN ERROR OCCURRED! {e}"
    return completion

def get_single_answer_hf(model, tokenizer, prompt, device=device):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    gen = model.generate(
        input_ids,
        num_beams = 1
    )[0]
    return tokenizer.decode(gen, skip_special_tokens=True)

In [ ]:
# for hf pretrained, hf own trained and gpt-3.5: repeat:

In [37]:
lang = "en"  # "en" or "de"

In [ ]:
# gpt-3.5-turbo
answers = []
with open(f"full_stories_{lang}.json", "r") as f:
    full_stories = json.load(f)
for fs in tqdm(full_stories):
    completion = get_single_answer_gpt("gpt-3.5-turbo", fs["prompt"])
    fs["completion"] = completion
    answers.append(fs)
with open(f"answers_{lang}_gpt.json", "w") as f:
    json.dump(answers, f, indent="    ", ensure_ascii=False)

In [ ]:
# TS pretrained
model_id = "roneneldan/TinyStories-1M"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

answers = []
with open(f"full_stories_{lang}.json", "r") as f:
    full_stories = json.load(f)
for fs in tqdm(full_stories):
    completion = get_single_answer_hf(model, tokenizer, fs["prompt"], device=device)
    fs["completion"] = completion
    answers.append(fs)
with open(f"answers_{lang}_ts.json", "w") as f:
    json.dump(answers, f, indent="    ", ensure_ascii=False)

In [ ]:
# TS own
model_path = f"/content/drive/My Drive/genAI project/model_1m_{lang}"
if not os.path.exists(model_path):
    print("ERROR: MODEL PATH DOES NOT EXIST")
    raise NameError
model_id = "roneneldan/TinyStories-1M"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

answers = []
with open(f"full_stories_{lang}.json", "r") as f:
    full_stories = json.load(f)
for fs in tqdm(full_stories):
    completion = get_single_answer_hf(model, tokenizer, fs["prompt"], device=device)
    fs["completion"] = completion
    answers.append(fs)
with open(f"answers_{lang}_own.json", "w") as f:
    json.dump(answers, f, indent="    ", ensure_ascii=False)